In [1]:
from bs4 import BeautifulSoup

In [ ]:
dir = '../../'

In [ ]:
for file_name in tqdm(os.listdir(dir)):
    if not file_name.endswith(".xml"):
        continue
    with open(os.path.join(dir, file_name)) as f:
        xmls.append({"file_name": file_name,
                     "content": BeautifulSoup(f.read(), features="xml")})